# Оллама + OpenAI + Пајтон

## 1. Наведите име модела

Ако сте преузели другачији модел од "phi3:mini", промените вредност у ћелији испод. Та променљива ће се користити у коду кроз целу свеску.


In [ ]:
MODEL_NAME = "phi3:mini"

## 2. Подешавање Open AI клијента

Обично се OpenAI клијент користи са OpenAI.com или Azure OpenAI за интеракцију са великим језичким моделима.  
Међутим, може се користити и са Ollama, јер Ollama пружа OpenAI-компатибилан крајњи чвор на "http://localhost:11434/v1".


In [ ]:
%pip install openai

In [ ]:
import openai

client = openai.OpenAI(
    base_url="http://localhost:11434/v1",
    api_key="nokeyneeded",
)

## 3. Генерисање одговора у ћаскању

Сада можемо користити OpenAI SDK да генеришемо одговор за разговор. Овај захтев треба да генерише хаику о мачкама:


In [ ]:
response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Write a haiku about a hungry cat"},
    ],
)

print("Response:")
print(response.choices[0].message.content)


## 4. Инжењеринг упита

Прва порука која се шаље језичком моделу назива се "системска порука" или "системски упит", и она поставља општа упутства за модел.  
Можете да обезбедите сопствени прилагођени системски упит како бисте усмерили језички модел да генерише излаз на другачији начин.  
Измените `SYSTEM_MESSAGE` испод да одговара као ваш омиљени лик из познатог филма/ТВ серије, или пронађите инспирацију за друге системске упите на [Awesome ChatGPT Prompts](https://github.com/f/awesome-chatgpt-prompts?tab=readme-ov-file#prompts).

Када прилагодите системску поруку, поставите прво корисничко питање у `USER_MESSAGE`.


In [ ]:
SYSTEM_MESSAGE = """
I want you to act like Elmo from Sesame Street.
I want you to respond and answer like Elmo using the tone, manner and vocabulary that Elmo would use.
Do not write any explanations. Only answer like Elmo.
You must know all of the knowledge of Elmo, and nothing more.
"""

USER_MESSAGE = """
Hi Elmo, how are you doing today?
"""

response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": USER_MESSAGE},
    ],
)

print("Response:")
print(response.choices[0].message.content)


## 5. Примери са неколико питања

Још један начин да усмерите језички модел је да му пружите „неколико примера“, низ питања и одговора који показују како би требало да одговара.

Пример испод покушава да наведе језички модел да се понаша као асистент у настави, пружајући неколико примера питања и одговора које би асистент могао да да, а затим поставља моделу питање које би студент могао да постави.

Прво пробајте, а затим измените `SYSTEM_MESSAGE`, `EXAMPLES` и `USER_MESSAGE` за нови сценарио.


In [ ]:
SYSTEM_MESSAGE = """
You are a helpful assistant that helps students with their homework.
Instead of providing the full answer, you respond with a hint or a clue.
"""

EXAMPLES = [
    (
        "What is the capital of France?",
        "Can you remember the name of the city that is known for the Eiffel Tower?"
    ),
    (
        "What is the square root of 144?",
        "What number multiplied by itself equals 144?"
    ),
    (   "What is the atomic number of oxygen?",
        "How many protons does an oxygen atom have?"
    ),
]

USER_MESSAGE = "What is the largest planet in our solar system?"


response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": EXAMPLES[0][0]},
        {"role": "assistant", "content": EXAMPLES[0][1]},
        {"role": "user", "content": EXAMPLES[1][0]},
        {"role": "assistant", "content": EXAMPLES[1][1]},
        {"role": "user", "content": EXAMPLES[2][0]},
        {"role": "assistant", "content": EXAMPLES[2][1]},
        {"role": "user", "content": USER_MESSAGE},
    ],
)


print("Response:")
print(response.choices[0].message.content)

## 6. Претраживачки проширена генерација

RAG (Претраживачки проширена генерација) је техника која омогућава језичком моделу да тачно одговара на питања за одређену област, тако што прво проналази релевантне информације из извора знања, а затим генерише одговор на основу тих информација.

Обезбедили смо локалну CSV датотеку са подацима о хибридним аутомобилима. Код испод чита CSV датотеку, тражи подударности са питањем корисника, а затим генерише одговор на основу пронађених информација. Имајте на уму да ће ово трајати дуже него било који од претходних примера, јер шаље више података моделу. Ако приметите да одговор и даље није заснован на подацима, можете пробати системско инжењерство или друге моделе. Генерално, RAG је ефикаснији са већим моделима или са фино подешеним верзијама SLM-ова.


In [ ]:
import csv

SYSTEM_MESSAGE = """
You are a helpful assistant that answers questions about cars based off a hybrid car data set.
You must use the data set to answer the questions, you should not provide any information that is not in the provided sources.
"""

USER_MESSAGE = "how fast is a prius?"

# Open the CSV and store in a list
with open("hybrid.csv", "r") as file:
    reader = csv.reader(file)
    rows = list(reader)

# Normalize the user question to replace punctuation and make lowercase
normalized_message = USER_MESSAGE.lower().replace("?", "").replace("(", " ").replace(")", " ")

# Search the CSV for user question using very naive search
words = normalized_message.split()
matches = []
for row in rows[1:]:
    # if the word matches any word in row, add the row to the matches
    if any(word in row[0].lower().split() for word in words) or any(word in row[5].lower().split() for word in words):
        matches.append(row)

# Format as a markdown table, since language models understand markdown
matches_table = " | ".join(rows[0]) + "\n" + " | ".join(" --- " for _ in range(len(rows[0]))) + "\n"
matches_table += "\n".join(" | ".join(row) for row in matches)
print(f"Found {len(matches)} matches:")
print(matches_table)

# Now we can use the matches to generate a response
response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": USER_MESSAGE + "\nSources: " + matches_table},
    ],
)

print("Response:")
print(response.choices[0].message.content)


---

**Одрицање од одговорности**:  
Овај документ је преведен коришћењем услуге за превођење помоћу вештачке интелигенције [Co-op Translator](https://github.com/Azure/co-op-translator). Иако се трудимо да превод буде тачан, молимо вас да имате у виду да аутоматизовани преводи могу садржати грешке или нетачности. Оригинални документ на његовом изворном језику треба сматрати меродавним извором. За критичне информације препоручује се професионални превод од стране људи. Не преузимамо одговорност за било каква погрешна тумачења или неспоразуме који могу настати услед коришћења овог превода.
